In [ ]:
import matplotlib.pyplot as plt
import datetime
import tilemapbase as tmb
import numpy as np
import pandas as pd
from pluma.stream.georeference import Georeference
from pluma.stream.ubx import _UBX_MSGIDS
from pluma.schema import Dataset

## Notebook plotting
%matplotlib inline
plt.style.use('ggplot')

## Figure export parameters
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
import matplotlib as mpl
mpl.rcParams.update(new_rc_params)

In [ ]:
stream_root_folder = r"s3://emotional-cities/data/nrg/poc-v1/TechnicalBenchmarks/TemporalBenchmarkSyncronizeOn2023_04_04_10_12_18/"
from benchmark_schemas import build_benchmarkschema_enobioeeg

dataset = Dataset(
    stream_root_folder,
    datasetlabel="EnobioEeg_Benchmark",
    georeference= Georeference(),
    schema=build_benchmarkschema_enobioeeg)
dataset.populate_streams(autoload=True)


In [ ]:
time_markers = dataset.streams.EEG.server_lsl_marker
time_markers[:100]

In [ ]:
# Get the synch pulse from the BioData "Set" stream
bit_mask = 0x03
rising_edge_events = dataset.streams.BioData.Set.data[dataset.streams.BioData.Set.data & bit_mask > 0]
rising_edge_events

## load eeg raw data
raw_time = dataset.streams.EEG.data.np_time
raw_markers = dataset.streams.EEG.data.np_markers
raw_eeg = dataset.streams.EEG.data.np_eeg[:,1]

time_markers = dataset.streams.EEG.server_lsl_marker

threshold = -50 # au EEG units
refractory_period = 2 # seconds
refractory_samples = refractory_period * dataset.streams.EEG.data.fs # samples

from scipy import signal

b,a = signal.butter(3, (1,30), 'bandpass', fs=dataset.streams.EEG.data.fs)
filtered_eeg = signal.filtfilt(b, a, raw_eeg)
digitized_eeg = (filtered_eeg < threshold).astype(float)
candidate_cross = np.where(np.diff(digitized_eeg) == 1)[0]
candidate_cross = candidate_cross[candidate_cross > 1*dataset.streams.EEG.data.fs]
candidate_cross = candidate_cross[np.diff(candidate_cross, prepend=np.inf) > refractory_samples]

plt.figure(figsize= (9,6))
plt.plot(raw_time, filtered_eeg)
plt.scatter(raw_time[candidate_cross], filtered_eeg[candidate_cross], c='g')
plt.xlim((20,200))
plt.ylim((-200,200))
plt.xlabel("Time (s)")
plt.ylabel("A.U. EEG units")
plt.show()

ttl_time = raw_time[candidate_cross]
_crop = np.min([len(ttl_time), len(time_markers)])
delta_ttl_to_lsl = (ttl_time[:_crop] - time_markers["EegTimestamp"][:_crop].values) * 1e3


plt.figure(figsize= (7,4))
plt.hist(delta_ttl_to_lsl, bins = 25, color= "Teal")
plt.xlabel('Latency (ms)')
plt.ylabel('Counts')
plt.title(f'EEG_tag to EEG_ttl latency benchmark\n$\mu = \
    {np.nanmean(delta_ttl_to_lsl):.2f}, \sigma = \
        {np.nanstd(delta_ttl_to_lsl):.2f},$ \
            [min:max] = [{np.nanmin(delta_ttl_to_lsl):.2f}:{np.nanmax(delta_ttl_to_lsl):.2f}]')
plt.show()


plt.figure(figsize= (7,4))
plt.plot(delta_ttl_to_lsl)
plt.title("Latency vs marker index")
plt.xlabel('Marker index')
plt.ylabel('Latency (ms)')
plt.show()

In [ ]:
## Check if the LSL server timestamp and Eeg timestamp drift in relationship to each other
plt.figure()
_delta = time_markers["LslTimestamp"].diff().values - time_markers["EegTimestamp"].diff()
plt.plot(_delta)
plt.hlines(np.nanmean(_delta), 0, len(_delta), 'black')
plt.xlabel("Marker index")
plt.ylabel("Delta time (s)")
plt.title("lsl_server vs eeg timestamps")
plt.show()

In [ ]:
## Check if the LSL server timestamp and harp DO drift in relationship to each other
server_lsl = dataset.streams.EEG.server_lsl_marker

plt.figure()
_delta = np.diff(server_lsl["LslTimestamp"].values) - np.diff(rising_edge_events.index.values).astype(float) / 1e9
plt.plot(_delta)
plt.hlines(np.nanmean(_delta), 0, len(_delta), 'black')
plt.xlabel("Marker index")
plt.ylabel("Delta time (s)")
plt.title("lsl_server vs ttl_harp timestamps")
plt.show()

In [ ]:
from pluma.io.harp import _HARP_T0
from sklearn.linear_model import LinearRegression

valid_samples = pd.notna(server_lsl["EegTimestamp"].values) & pd.notna(server_lsl["Seconds"].values)
raw_harp_time = (server_lsl["Seconds"].values - np.datetime64(_HARP_T0)) / np.timedelta64(1, 's')
eeg_time = server_lsl["EegTimestamp"].values
eeg_time = eeg_time.reshape(-1,1)
raw_harp_time = raw_harp_time.reshape(-1,1)

model = LinearRegression(fit_intercept=True).fit(eeg_time[valid_samples], raw_harp_time[valid_samples])
r2 = model.score(eeg_time[valid_samples], raw_harp_time[valid_samples])
print(model.coef_)
print(r2)
